# 준비

## 목표
* 웹 크롤링의 전반적인 원리와 절차를 이해한다.
* requests와 beautifulsoup4 라이브러리를 이용한 웹 크롤링 기법을 익힌다.
* splinter 라이브러리와 웹 드라이버를 이용한 실제 브라우저 조작 방법을 살펴본다.
* twython 라이브러리를 이용한 트윗 수집 방법을 살펴본다.

## 사용 라이브러리
위에서 언급한 대로 이번 강의에서 사용하는 주요 라이브러리들은 다음과 같다.

| 명칭 | 기능 | 배포판 포함 여부 | 설치 도구 | 홈페이지 |
|:-----|:-----|:--------------:|:---------:|:-----------|
| ujson | 고속 JSON 인코딩/디코딩 | 미포함 | conda | <https://github.com/esnme/ultrajson> |
| ordered-set | 원소 추가 순서 유지 세트 | 미포함 | pip | <https://github.com/LuminosoInsight/ordered-set> |
| requests | 간편한 HTTP 통신 | 아나콘다 |  | <http://docs.python-requests.org> |
| beautifulsoup4 | 견고한 HTML/XML 파싱 | 아나콘다 | | <https://www.crummy.com/software/BeautifulSoup/bs4/doc> |
| twython | 간편 트위터 API 사용 | 미포함 | pip | <https://twython.readthedocs.io> |
| splinter | 간편한 웹 드라이버 구동 | 미포함 | pip | <https://splinter.readthedocs.io> |

ujson 모듈을 아나콘다 파이썬의 표준 패키지 관리 도구인 conda를 이용하여 설치해 보자. conda는 명령행 도구이므로 다음 그림과 같이 [메뉴]-[Anaconda3 (64bit)]-[Anaconda Prompt]를 실행하여 아나콘다 명령행 창을 실행한다.

![응용 프로그래 메뉴를 통한 아나콘다 명령행 창 실행](figs/prg-ana-cmd.png)

이어서 다음과 같이 `conda install ujson` 명령을 입력하고 [ENTER]를 눌러 명령을 실행하고 물음에 `y`로 답하면 패키지 설치가 진행된다.

![conda를 이용한 ujson 모듈 설치](figs/ana-cmd-conda.png)

설치가 끝나면 아나콘다 명령행 창을 닫는다.

conda는 위와 같이 명령행 창을 이용하여 사용하는 것이 기본이지만 다음 그림과 같이 응용 프로그램 메뉴에서 Anaconda Navigator를 실행하여 그래픽 사용자 인터페이스를 통해 conda를 사용하는 방법도 있다.

![아나콘다 네비게이터를 통한 패키지 설치](figs/ana-nav.png)

아나콘다 파이썬 저장소에서 지원하는 않는 패키지는 표준 파이썬 패키지 관리 도구인 `pip`을 이용하여 설치해야 한다. 사용 방법은 `conda`와 마찬가지이다.

# 웹 크롤링 개관
## 웹 크롤링이란?
웹 크롤링(web crawling)은 웹을 통하여 서비스되는 문서를 자동으로 수집하여 저장하는 과정을 말한다. 경우에 따라 문서 전체를 저장하기도 하고 문서에서 필요한 부분만을 추출하여 저장하기도 한다. 웹 스크래이핑(web scraping)이라고 부르기도 한다. 웹 크롤링을 위해 만드는 프로그램은 웹 크롤러(web crawler), 웹 스크래이퍼(web scraper), 웹 스파이더(web spider), 웹 로봇(web robot) 등으로 부른다. 구글이나 네이버와 같이 검색 서비스의 제공을 위해 웹 문서의 자동 수집에 이용하는 로봇이 대표적인 웹 크롤러이다.

>웹 스크래이핑은 웹 문서의 저장만을, 웹 크롤링은 정보 추출까지 이루어지는 경우로 구분하기도 한다.

## 웹 크롤링의 원리
웹 크롤링의 기본적인 동작 원리는 인간 사용자를 위한 웹 클라이언트인 **웹 브라우저(인터넷 익스플로러, 파이어폭스, 크롬 등)의 동작을 다른 소프트웨어로 흉내내도록 하는 것**이다. 크롤링을 위해서는 첫 접근 문서에서 다른 문서로의 링크를 따라 이동하는 기능이 반드시 필요하다. 비교적 간단한 웹 사이트의 경우에는 크롤러가 읽은 **HTML 문서를 분석**하여 **링크들을 추출하고 그 링크를 따라가도록** 한다. 구조가 복잡하거나 폼(form) 인터페이스, 연속 스크롤(continuous scroll) 등과 같이 **동적으로 콘텐츠가 생성되는 웹 사이트의 경우에는 헤드레스, 혹은 실제 웹 브라우저를 크롤러를 통해 구동**해야 하기도 한다. 이러한 웹 사이트의 수집 절차는 일반화하기 어려운 경우가 대부분이다.

>헤드레스(headless) 브라우저란 화면 상에 표시되는 인터페이스가 없이 메모리 상에서만 동작하는 브라우저를 말한다. 웹 크롤링, 자동 스크린샷 찍기 등을 위해 사용한다. 현재 가장 널리 사용되는 헤드레스 브라우저는 PhantomJS(<http://phantomjs.org>)이다. 한편 최근 크롬 브라우저에 헤드리스 기능이 추가(<https://developers.google.com/web/updates/2017/04/headless-chrome>)되었다.

앞서 언급한 HTML 문서의 분석은 HTML 문서에서 HTML 요소(HTML elements) 또는 웹 요소(Web elements)를 구분하고 이들의 계층적 구조를 문서 객체 모델(Document Object Model: DOM)으로 형성하는 과정이다. 다음 예를 보자.

```HTML
<html>
<head>
    <title>HTML 파일 예제</title>
</head>
<body>
    <h1>이것은 제목입니다.</h1>
    <p name="my_para">
        이것은 
        <a class="my_link" href="http://www.fastcampus.co.kr/data_camp_textmining/">링크</a>를 
        포함한 단락입니다.
    </p>
</body>
</html>
```

![예제 문서로부터 생성된 DOM](figs/dom.png)

웹 서버로부터 전송된 HTML 문서로부터 위와 같이 DOM을 생성한 이후에는 DOM을 구성하고 있는 HTML 요소 가운데 특정 요소를 선택할 수 있으며 컨텐트 문자열을 포함한 해당 요소의 하위 요소를 추출하는 것도 가능하다. 웹 브라우저는 이렇게 선택된 특정 요소에 대하여 특화된 렌더링(화면 표시)을 적용할 수 있으며, 어떤 요소들은 마우스 단추 클릭 등의 사용자 행위에 반응하도록 한다. 특정 HTML 요소를 선택할 때에는 해당 요소를 가리키는 태그(tag), 속성(attribute), 해당 요소의 절대적, 상대적 위치 등을 복합적으로 이용한다. 웹 크롤러는 웹 브라우저를 흉내내어 DOM을 생성한 뒤 필요한 요소를 추출하거나 사용자 행위를 발생시켜 동적으로 컨텐트를 생성한다.

한편 다양한 정보 제공을 위한 API 형태의 웹 서비스를 제공되는 사이트의 경우에는 문서와 정보의 수집을 위하여 API를 이용하는 것이 바람직하다. 예를 들어, 트위터에서는 키워드에 의한 문서 탐색, 특정 사용자의 팔로워 얻기 등을 지원하는 API 서비스를 제공한다. 그러므로 트윗을 수집하고자 하는 경우에는 웹 크롤링 방식이 아닌 API 서비스를 이용하는 것이 옳은 접근이다.

>API(Application Programming Interface)란 운영체제나 특정 프로그래밍 언어에서 사용할 수 있는 기본적인 기능을 라이브러리 등의 형태로 제공하는 것을 말한다. 최근에는 많은 사이트들이 다양한 정보를 제공하는 웹 서비스를 API의 형태로 제공하고 있다.

## 웹 크롤링의 절차
웹 크롤링은 대체로 다음의 순서로 진행한다. 간단한 크롤링의 경우에는 아래의 세 단계를 하나로 합치기도 하고 경우에 따라 두 번째와 세 번째 단계를 하나로 합치기도 한다.

### 문서 URL의 수집과 저장
문서, 이미지, 동영상, 음악 등 그 형태에 관련 없이 웹에서 접근 가능한 모든 자원은 고유한 URL(Uniform Resource Locator)에 의해 접근이 가능하다. 특정 자원을 가리키는 URL이 바뀌거나 없어지는 경우가 있을 수 있지만 원칙적으로 URL을 알고 있으면 나중에라도 그 자원에 접근할 있다. 그러므로 웹 크롤링의 첫 번째 단계는 웹 문서를 가리키는 URL을 수집하여 저장하는 것이다. 앞서 언급한 바와 같이 간단한 정적 웹 사이트의 경우에는 URL을 얻는 일이 비교적 간단하지만 동적 웹 사이트의 경우에는 URL을 알아내기가 매주 힘든 경우도 있다.

### 문서의 수집과 저장
이 단계에서는 위의 단계에서 수집한 문서 URL들을 읽어서 문서를 수집하여 저장한다. 동적 웹 사이트의 경우에는 URL을 통한 문서의 수집이 불가능하고 웹 페이지의 특정 요소를 누른다거나 웹 브라우저에서 자바스크립트가 실행되어야 문서에 접근이 가능한 경우도 있다. 단순해 보이는 단계이지만 대규모 크롤링 프로젝트에서는 오류의 대처, 대용량 문서의 효율적인 저장, 작업 이력 관리 등 고려해야 할 사안이 매우 많다.

### 문서 단순화와 구조화
웹 문서는 대체로 기본적인 텍스트에 구조화, 다양한 장식 등을 위한 태그가 부착된 HTML 형식의 문서이다. 그런데 일반적으로 텍스트 마이닝에서 필요한 것은 태그 등 부가 표식이 모두 제거된 텍스트이므로 이 단계에서 HTML, 자바스크립트 등 순수 텍스트가 아닌 요소를 잘라낸다. 경우에 따라서는 반복적인 머릿말이나 꼬릿말, 광고 등 본문의 요소가 아닌 텍스트를 제외하는 작업을 해야 하기도 한다.

## 고려 사항
웹 크롤링은 타인 혹은 타조직이 생성한 콘텐츠에 일반적이지 않은 방법으로 접근하는 일이므로 여러 가지 면으로 민감한 작업이다. 현실적으로 볼 때 인간 사용자가 아닌 크롤러의 접근을 환영하는 웹 사이트는 만나기 어렵다. 그러므로 최소한 다음의 사항들은 면밀히 검토하고 정책을 세워야 한다.

* 서비스 이용 약관: 개인 웹 사이트를 제외한 대부분의 웹 사이트들은 서비스 이용 약관을 웹 사이트에 게시한다. 이 약관에는 서비스 제공자의 권리와 책임의 한계, 서비스 이용자의 권리 등이 명시되어 있다.
* 저작권 보호: 이는 웹 크롤링 자체의 고려 사항이라기보다는 주로 크롤링 이후의 수집된 문서와 자료의 이용에 관련된 것이지만 크롤링을 통해 문서와 자료를 저장하는 행위 역시 저작권 보호에 위배되는 행위일 수 있으므로 주의를 기울여야 한다.
* 개인정보 보호: 이 사안 역시 크롤링된 문서의 자료의 활용에 관련된 것인데 최근 특히 많은 주목을 받는 사안이므로 특별히 관심을 두는 것이 좋다.
* 크롤링 예절: 앞서 언급한 바와 같이 크롤링에 의한 웹 자원으로의 접근은 인간에 의한 접근과는 매우 다르므로 웹 사이트에 의도하지 않은 영향을 끼칠 가능성이 있다. 그러므로 접근 시간 간격 등 조정 가능한 모든 요소를 면밀히 조정하여 해당 웹 사이트가 정상 상태로 운영되도록 해야 한다.

>웹 사이트에 따라 크롤러, 즉 로봇의 접근 범위를 규정하는 `robots.txt`가 주어지는 경우가 있다. 이 파일의 내용은 강제적이지는 않으나 따르는 것이 원칙이다. 예를 들어 네이트 뉴스 사이트에는 <http://news.nate.com/robots.txt>에 해당 파일이 존재한다.

# 웹 크롤링 기초 예제

앞 강의에서 자료로 사용한 이청준 작가의 "벌레 이야기" 텍스트를 웹에서 크롤하는 간단한 예제를 살펴보자. 목표 URL은 웹 검색을 통해 얻은 것이다.

In [2]:
import requests
from bs4 import BeautifulSoup

# 목표 url
url = "http://cafe.daum.net/_c21_/bbs_search_read?grpid=MWoV&fldid=IGhr&datanum=1805"
# 웹 서버에 요청 보내고 응답 받기
resp = requests.get(url)
# 전송된 HTML 문서 얻기
html = resp.text

# HTML 문서를 DOM으로 변환하기
soup = BeautifulSoup(html, "lxml")

#help(BeautifulSoup)


In [3]:
# 목표 HTML 요소 선택
elem = soup.find("xmp")   # HTML 객체, BS객체 

In [4]:
# 목표 텍스트 추출
text = elem.get_text()   # 하나의 string으로...

with open("../data/crawling/worm2.txt", "w", encoding="utf-8") as text_file:
    print(text, file=text_file)

In [13]:
elem.contents   # 리스트로...

[' ',
 <p> </p>,
 '\n',
 <p align="center"><strong><font size="5">&lt;벌레이야기&gt;</font></strong></p>,
 '\n',
 <p> </p>,
 '\n',
 <p>1</p>,
 '\n',
 <p>아내는 알암이의 돌연스런 가출이 유괴에 의한 실종으로 확실시되고 난 다음에도 한동안은 악착스럽게 자신을 잘 견뎌 나가고 있었다. 그것은 아이가 어쩌면 행여 무사히 되돌아오게 될지도 모른다는 간절한 희망과, 녀석에게 마지막 불행한 일이 생기기 전에 어떻게든지 놈을 다시 찾아내고 말겠다는 어미로서의 강인한 의지와 기원 때문인 것 같았다.</p>,
 '\n',
 <p>지난해 5월 초. 어느 날 알암이가 학교에서 돌아올 시각이 훨씬 지나도록 귀가를 안 했다.<br/>달포 전에 갓 초등학교 4학년을 올라간 녀석은 학교에서 돌아오는 길로 곧장 다시 동네 상가에 있는 주산 학원을 나가야 했다. 우리가 부러 시킨 일이 아니라 녀석이 좋아서 쫓아다니는 곳이었다.<br/>다리가 한 쪽 불편한 때문이었을까. 제 어미 마흔 가까이에 얻어난 녀석이 어릴 적부터 성미가 남달리 유순했다. 유순한 정도를 지나 내숭스러워 보일 만큼 나약하고 조용했다. 어려서부터 통 집 밖엘 나가 노는 일이 없었다. 동네 아이들과도 어울리려 하질 않았다. 집 안에서만 혼자 하얗게 자라갔다. 혼자서 무슨 특별한 놀이를 탐구하는 일도 없었다. 무슨 일에도 취미를 못 붙이고 애어른처럼 그저 방 안에만 틀어박혀 적막스런 나날을 지내고 있었다. 녀석의 몸짓이나 말투까지도 그렇게 조용조용 조심스럽기만 하였다.<br/>초등학교엘 입학하고 나서도 마찬가지였다. 태어날 때부터의 불구에 이력이 붙은 우리 부부는 말할 것도 없었고, 녀석의 담임 반 선생님까지도 각별한 주의를 기울여 살폈지만, 녀석에겐 전혀 별다른 변화의 기색이 나타나질 않았다. 친구를 가까이 사귀는 일이나, 어떤 학과목에 특별히 취미를 붙여가는 낌새가 전혀 없었다. 특별한 취미는 없어 하면서도 학과목

In [14]:
text

' \xa0\n<벌레이야기>\n\xa0\n1\n아내는 알암이의 돌연스런 가출이 유괴에 의한 실종으로 확실시되고 난 다음에도 한동안은 악착스럽게 자신을 잘 견뎌 나가고 있었다. 그것은 아이가 어쩌면 행여 무사히 되돌아오게 될지도 모른다는 간절한 희망과, 녀석에게 마지막 불행한 일이 생기기 전에 어떻게든지 놈을 다시 찾아내고 말겠다는 어미로서의 강인한 의지와 기원 때문인 것 같았다.\n지난해 5월 초. 어느 날 알암이가 학교에서 돌아올 시각이 훨씬 지나도록 귀가를 안 했다.달포 전에 갓 초등학교 4학년을 올라간 녀석은 학교에서 돌아오는 길로 곧장 다시 동네 상가에 있는 주산 학원을 나가야 했다. 우리가 부러 시킨 일이 아니라 녀석이 좋아서 쫓아다니는 곳이었다.다리가 한 쪽 불편한 때문이었을까. 제 어미 마흔 가까이에 얻어난 녀석이 어릴 적부터 성미가 남달리 유순했다. 유순한 정도를 지나 내숭스러워 보일 만큼 나약하고 조용했다. 어려서부터 통 집 밖엘 나가 노는 일이 없었다. 동네 아이들과도 어울리려 하질 않았다. 집 안에서만 혼자 하얗게 자라갔다. 혼자서 무슨 특별한 놀이를 탐구하는 일도 없었다. 무슨 일에도 취미를 못 붙이고 애어른처럼 그저 방 안에만 틀어박혀 적막스런 나날을 지내고 있었다. 녀석의 몸짓이나 말투까지도 그렇게 조용조용 조심스럽기만 하였다.초등학교엘 입학하고 나서도 마찬가지였다. 태어날 때부터의 불구에 이력이 붙은 우리 부부는 말할 것도 없었고, 녀석의 담임 반 선생님까지도 각별한 주의를 기울여 살폈지만, 녀석에겐 전혀 별다른 변화의 기색이 나타나질 않았다. 친구를 가까이 사귀는 일이나, 어떤 학과목에 특별히 취미를 붙여가는 낌새가 전혀 없었다. 특별한 취미는 없어 하면서도 학과목 성적만은 또 전체적으로 고루 상급에 속할 만큼 제 할 일은 제대로 하고 다니는 녀석이었다.그런데 지낸해 봄, 녀석이 4학년엘 올라가고 나서였다. 이때까지 전혀 어떤 특별 활동 시간에도 관심을 보이지 않던 녀석이 이번엔 누가 권하지 않았는데도 제물에 새로 생긴 주산반엘 들

위의 소스를 분석해 보자.

* requests 모듈의 `get()` 함수는 URL을 인자로 받아 해당 URL이 가리키는 웹 문서를 웹 서버에 요청한다. 웹 서버는 이 요청에 대한 응답을 돌려준다.
* 응답 객체의 `text` 속성은 웹 응답에 포함되어 전송된 텍스트를 담고 있다. 보통은 HTML 텍스트이지만 다른 형식의 플레인 텍스트일 수도 있다.
* BeautifulSoup 클래스는 객체를 초기화할 때에 HTML 텍스트와 파싱에 사용할 파서 라이브러리의 이름을 인자로 넘긴다. 분석 속도와 견고성이 적절한 균형을 이룬 lxml이 널리 쓰인다.
* BeautifulSoup 객체는 HTML 요소의 탐색과 조작을 지원하는 다양한 메소드와 속성을 지원한다. `find()` 메소드는 인자로 주어진 태그가 가리키는 HTML 요소를 돌려준다. 유의할 것은 BeautifulSoup 객체와 HTML 요소 객체는 동일한 것이라는 점이다.
* HTML 요소 객체의 `get_text()` 메소드는 해당 요소의 하위 요소들 가운데 컨텐트 문자열들을 추출하여 돌려준다.

BeautifulSoup 클래스는 HTML 요소의 탐색에 CSS 선택자(Cascading Style Sheet selector)를 지원한다. W3 Schools 사이트(<https://www.w3schools.com/cssref/css_selectors.asp>)에서 자세한 CSS 선택자의 목록을 볼 수 있다. 특정 HTML 요소의 CSS 선택자는 웹 브라우저에서 지원하는 개발자 도구(혹은 검사 도구)를 이용하여 확인할 수 있다. 다음 그림에서 위의 예제의 목표 HTML 텍스트에서 소설의 본문을 추출하기 위해 접근이 간편한 HTML 요소인 `xmp` 요소를 확인하는 것을 볼 수 있다.

![목표 텍스트 추출을 위한 HTML 요소 확인](figs/worm-css.png)

# 실전 웹 크롤링

전형적인 한국형 게시판 중심의 커뮤니티 사이트인 캐토리(<http://catory.kr>)를 대상으로 실전 웹 크롤링 과정을 구현해 보자.

## 사이트 구조 파악
캐토리는 몇 개의 상위 메뉴와 하위 메뉴로 구성되어 있으며, 각 하위 메뉴는 개별 게시판에 해당한다. 아래의 그림은 메인 화면에서 [고양이뉴스]-[고양이뉴스]의 경로를 거쳐 "고양이뉴스" 게시판에 들어간 화면이다.

![캐토리 사이트의 고양이뉴스 게시판](figs/catory-bbs.png)

위의 화면에서 볼 수 있는 것처럼 각 게시판의 페이지는 일차적으로 일정한 개수의 게시물의 제목 등과 해당 게시물로의 링크를 목록으로 보여주는 목록 페이지로 구성된다. 게시물 제목, 사진 등의 링크를 클릭하면 개별 게시물의 표시 화면으로 이동한다.

![캐토리 사이트의 게시물 표시 화면](figs/catory-post.png)

위의 그림에서 볼 수 있는 것처럼 개별 게시물 표시 화면에서는 게시물의 제목, 작성자, 게시 날짜와 시간, 본문 등이 별도의 HTML 요소에 담겨 분리되어 표시된다.

이상을 요약하면 캐토리의 구조는 **메인 페이지 -- 게시판 목록 페이지 (페이지 이동) -- 게시물 페이지**로 되어 있다. 앞서 설명한 바와 같이 웹 크롤링의 첫 번째 절차는 목표 URL을 파악하는 것이다. 위의 사이트 구조 파악을 통해 파악된 주요 URL들은 다음과 같다.

* 메인 페이지: <http://catory.kr>
* 고양이뉴스 게시글 목록 1 페이지: <http://catory.kr/index.php?mid=fcNews&category=143>
* 고양이뉴스 게시글 목록 2 페이지: <http://catory.kr/index.php?mid=fcNews&category=143&page=2>
* 게시물 페이지: <http://catory.kr/index.php?mid=fcNews&category=143&document_srl=10623>

위의 URL을 통해 다음과 같은 사실을 파악할 수 있다.

* 주메뉴는 `mid=fcNews`와 같이 `mid` 변수에 `fcNews` 등의 값을 할당하여 지정한다.
* 각 게시판의 목록 페이지는 `category=143`과 같이 `category` 변수에 `143` 등의 값을 할당하여 지정한다.
* 목록 페이지의 페이지 이동은 `page=2`와 같이 `page` 변수에 `2`와 같이 페이지 번호를 할당하여 이루어진다.
* 개별 게시물은 `document_srl=10623`와 같이 `document_srl` 변수에 `10623`과 같이 게시물 번호를 할당하여 지정한다.

## 웹 크롤링 전략 수립
캐토리 사이트의 게시물 수집 전략을 다음과 같이 수립한다.

* 개별 게시판의 게시글 목록 페이지 주소를 수집 시작점으로 한다. 게시판으로 이루어진 사이트의 게시물 수집은 게시판 단위로 수행하는 것이 효율적이다.
* 게시물 목록 페이지를 페이지별로 이동하면서 게시물 URL을 추출한다.
* 추출한 게시물 URL들에 접근하여 게시물 제목, 작성자, 날짜, 본문을 추출한다.
* 추출은 게시물 요소를 JSON 형식으로 출력 파일에 저장한다.

위의 전략을 실행에 옮기기 위해서는 먼저 정보 추출 대상 HTML 요소들의 CSS 선택자를 파악해야 한다. 앞서 보인 기초 예제에서와 같이 웹 브라우저의 개발자 도구를 사용해야 한다. 또한 잊지 말아야 하는 것은 게시물 목록 페이지의 페이지 이동 중지 조건이다. 즉, 끝 페이지를 판단하는 기준을 파악해야 한다. 캐토리의 경우는 아래 그림에서 보는 것과 같이 간단한 실험을 통해 페이지 이동 종료 조건을 알 수 있다.

![캐토리 페이지 이동 종료 조건](figs/catory-paging.png)

캐토리의 게시물 목록 페이지 이동 종료 조건은 HTML 텍스트에 `게시물이 없습니다.`라는 문자열이 포함 여부로 정해진다.

위에서 세운 크롤링 전략을 코드로 구현하기 전에 의사 코드로 표현해 보자. 편의상 앞서 살펴본 "고양이뉴스" 게시판의 게시글만 수집하기로 하자.

1. 게시물 URL을 담을 세트를 준비한다.
1. 페이지 번호를 1로 설정한다.
1. 페이지 번호를 증가시키며 아래의 동작을 반복한다. 
    1. 게시물 목록 페이지 URL을 생성한다.
    1. 생성된 URL로 접속하여 HTML 텍스트를 얻는다.
    1. HTML 텍스트에서 페이지가 페이지 이동 종료 조건에 부합하면 반복을 멈춘다.
    1. soup 객체를 생성한다.
    1. soup 객체에서 게시물 URL을 추출하여 준비한 세트에 추가한다.
    1. 페이지 번호를 1 증가시킨다.
1. 출력 파일을 생성한다.
1. 모든 게시물 URL에 대해서 아래의 동작을 반복한다.
    1. 게시물 URL에 접속하여 HTML 텍스트를 얻는다.
    1. soup 객체를 생성한다.
    1. soup 객체에서 제목을 추출한다.
    1. soup 객체에서 작성자를 추출한다.
    1. soup 객체에서 작성일시를 추출한다.
    1. soup 객체에서 본문을 추출한다.
    1. 제목, 작성자, 작성일시, 본문, URL을 JSON 호환 객체로 만든다.
    1. 출력 파일에 JSON 문자열을 기록한다.
1. 출력 파일을 닫는다.

## 웹 크롤러 구현
앞서 보인 의사 코드를 코드로 구현하면 다음과 같다.

In [ ]:
help(print)

In [ ]:
"""
catory.kr 고양이뉴스 게시판 게시물 수집
"""

import requests
from bs4 import BeautifulSoup
from ordered_set import OrderedSet
import ujson
import re
import time


USER_AGENT = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 " + \
    "(KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
HEADERS = {"User-Agent": USER_AGENT}
TOP_URL = "http://catory.kr"
LIST_PAGE_TMPL = "http://catory.kr/index.php?mid=fcNews&category=143&page={}"
PAGING_DONE_PAT = "게시물이 없습니다."
PAUSE_INT = 1.5
OUTPUT_FILE_NAME = "../data/crawling/catory-cat-news.txt"


def collect_post_urls():
    """게시물 URL들을 수집하여 돌려준다."""

    print("Collecting post urls.", flush=True)  #flush 기능이 뭐지? 버퍼하지 말고 바로 출력하라고... 
    post_urls = OrderedSet()
    page_num = 1
    
    while True:
        print("Accessing post list page {}.".format(page_num))
        list_page_url = gen_list_page_url(page_num)
        html = get_html(list_page_url)
        
        if is_paging_done(html):
            break
            
        soup = get_soup(html)
            
        sub_post_urls = ext_post_urls(soup)
        post_urls.update(sub_post_urls)
        page_num += 1
            
    print("Done collecting post urls.", flush=True)
        
    return post_urls


def gen_list_page_url(page_num):
    """주어진 페이지 번호에 해당하는 목록 페이지의 URL을 생성하여 돌려준다."""
    
    list_page_url = LIST_PAGE_TMPL.format(page_num)
    
    return list_page_url


def get_html(url):
    """주어진 URL에 접속하여 HTML 텍스트를 얻어서 돌려준다."""
    
    response = requests.get(url, headers=HEADERS)
    html = response.text   
    pause()
    
    return html


def is_paging_done(text):
    """주어진 텍스트에 페이지 이동 종료 패턴이 있는지 여부를 판단하여 돌려준다."""
    
    if PAGING_DONE_PAT in text:
        return True
    
    return False


def get_soup(html):
    """주어진 HTML 텍스트를 분석하여 soup 객체를 만들어 돌려준다."""
    
    soup = BeautifulSoup(html, "lxml")
    
    return soup
    
    
def ext_post_urls(soup):
    """주어진 soup 객체에서 게시물 URL들을 추출하여 돌려준다."""
    
    outer_elem = soup.find("div", class_="board_list")
    post_urls = [(TOP_URL + elem["href"]) 
                 for elem in outer_elem.find_all("a", class_="hx")]
    
    return post_urls


def pause():
    """정해진 만큼의 시간 동안 쉰다."""
    
    time.sleep(PAUSE_INT)
    
    
def collect_posts(post_urls):
    """주어진 게시물 URL들에 접근하여 게시물들을 수집한다."""
    
    print("Collecting posts.")
    num_post_urls = len(post_urls)
    
    with open(OUTPUT_FILE_NAME, "w", encoding="utf-8") as output_file:
        for i, post_url in enumerate(post_urls, 1):
            print("Accessing post url {}/{}".format(i, num_post_urls),
                  flush=True)
            html = get_html(post_url)
            soup = get_soup(html)
            title = ext_title(soup)
            author = ext_author(soup)
            date_time = ext_date_time(soup)
            view_count = ext_view_count(soup)
            like_count = ext_like_count(soup)
            body = ext_body(soup)
            json_obj = compose_json(title, author, date_time, view_count, 
                                    like_count, body, post_url)
            write_json_post(output_file, json_obj)
            
    print("Done collecting posts.")
    
    
def ext_title(soup):
    """주어진 soup 객체로부터 제목을 추출하여 돌려준다."""
    
    elem = soup.find("h1", class_="noyebbs-head-title")
    title = elem.string.strip()
    
    return title
    
    
def ext_author(soup):
    """주어진 soup 객체로부터 저자를 추출하여 돌려준다."""
    
    elem = soup.find("span", class_="author")
    author = elem.text.strip()
    
    return author   


def ext_date_time(soup):
    """주어진 soup 객체로부터 게시 일시를 추출하여 돌려준다."""
    
    elem = soup.find("span", class_="time")
    date_time = elem.string.strip()
    
    return date_time   


def ext_view_count(soup):
    """주어진 soup 객체로부터 조회수를 추출하여 돌려준다."""
    
    elem = soup.find("span", class_="read_count")
    read_count = elem.string.strip()
    read_count = int(get_digits(read_count))
    
    return read_count  


def ext_like_count(soup):
    """주어진 soup 객체로부터 추천수를 추출하여 돌려준다."""
    
    elem = soup.find("span", class_="vote_count")
    vote_count = elem.string.strip()
    vote_count = int(get_digits(vote_count))
    
    return vote_count 


def get_digits(text):
    """주어진 문자열에서 수자들만 골라 돌려준다."""
    
    digit_chars = [char for char in text if char.isdigit()]
    digits = "".join(digit_chars)
    
    return digits


def ext_body(soup):
    """주어진 soup 객체로부터 본문을 추출하여 돌려준다."""
    
    elem = soup.find("div", class_=re.compile("^document_"))
    body = elem.text.strip()   
    
    return body


def compose_json(title, author, date_time, view_count, like_count, 
                 body, url):
    """주어진 게시물 요소들로부터 JSON 객체를 생성하여 돌려준다."""
    
    json_obj = {
        "title": title,
        "author": author,
        "date_time": date_time,
        "view_count": view_count,
        "like_count": like_count,
        "body": body,
        "url": url
    }
    
    return json_obj


def write_json_post(output_file, json_obj):
    """주어진 출력 파일에 JSON 문자열 게시물을 기록한다."""

    json_str = ujson.dumps(json_obj, ensure_ascii=False)
    print(json_str, file=output_file, flush=True)
    

def main():
    """catory.kr 고양이뉴스 게시판의 게시글을 수집하여 저장한다."""
    
    post_urls = collect_post_urls()
    collect_posts(post_urls)
    
#
# 실행
#

main()

위의 코드에서 특히 관심을 가지고 살펴볼 부분은 다음과 같다.

* `collect_post_urls()`
    - ordered-set 모듈의 OrderedSet 클래스를 이용하여 게시물 URL을 저장한다.
* `get_html()` 
    - requests 모듈의 `get()` 함수를 이용하여 HTTP 통신을 할 때에 헤더를 지정하는 방식으로 User-Agent를 사용한다.
    - time 모듈의 `sleep()` 함수를 이용하여 HTTP 요청 간 일정한 간격을 준다.
* `ext_post_urls()`
    - 경우에 따라 목표 HTML 요소에 바로 접근하기보다 적절한 바깥쪽 요소에 먼저 접근하는  것이 안전하다.
    - HTML 요소의 속성 값은 딕셔너리 형식으로 얻을 수 있다.
* `ext_body()`
    - CSS 선택자를 이용하여 목표 HTML 요소에 접근할 때에 정규 표현을 이용한 문자열 매치를 이용할 수 있다.
    - 또 다른 방법은 바로 직전, 혹은 다음 요소에 먼저 접근한 뒤 상대 위치에 따라 목표 요소에 접근하는 것이다.
* `write_json_post()`
    - `print()` 함수를 사용할 때에 `flush=True`로 지정하면 데이터를 버퍼링 없이 바로 출력한다. 출력 속도는 느려진다.

# 웹 브라우저의 조작

데이터 수집 대상이 되는 웹 사이트 가운데에는 목표 데이터를 얻을 수 있는 URL이 명시적으로 주어지 않거나 특정한 요소를 클릭하는 등의 동작이 있어야만 해당 데이터에 접근할 수 있는 경우가 있다. 또한 HTML 텍스트를 웹브라우저가 읽어들인 이후에 자바스크립트가 실행되어 최종 데이터가 생성되는 사이트들도 늘고 있다. 이와 같은 사이트의 데이터 수집을 위해서는 사용자 상호작용과 자바스크립트의 실행을 지원하는 실제 브라우저를 구동해야 한다.

파이썬에서 웹브라우저를 구동하는 방법에는 대표적으로 두 가지 방법이 있다. 첫 번째는 selenium 모듈의 웹드라이버를 이용하여 파이어폭스, 크롬 등의 실제 브라우저나 PhantomJS 등의 헤드레스 브라우저를 구동하는 것이고, 두 번째는 HTML 렌더링 라이브러리인 웹킷을 사용하는 것이다. 

이 강의에서는 selenium을 더 편하게 쓸 수 있도록 포장한 splinter 모듈을 이용하여 크롬 브라우저를 조작하여 네이버 포털에 접속하여 검색을 수행하는 방법을 보인다. 이 예제를 실행하려면 다음의 준비가 필요하다.

* `pip`을 이용하여 Splinter 모듈을 설치해야 한다.
* `https://sites.google.com/a/chromium.org/chromedriver/downloads`에서 제공하는 크롬 드라이버를 내려 받아 설치해야 한다.

In [ ]:
"""
크롬 브라우저를 스크립트로 조종하여 네이버 서비스를 이용한다.
"""

import getpass
import time
import splinter


HOME_URL = "http://www.naver.com"
LOGIN_URL = "https://nid.naver.com/nidlogin.login"
LOGOUT_URL = "http://nid.naver.com/nidlogin.logout?returl=http://www.naver.com"
SLEEP = 3
ID_CSS = "input#id.int"
PW_CSS = "input#pw.int"
LOGIN_BTN_CSS = "input.btn_global"
QUERY_CSS = "input#query.input_text"
SEARCH_BTN_CSS = "button#search_btn.sch_smit"
LOGOUT_BTN_CSS = "span.btn_inr"


def get_naver_user_id_pw():
    """네이버 사용자 ID와 암호를 입력받아 돌려준다."""

    user_id = input("Enter Naver user ID: ")
    user_pw = getpass.getpass("Enter Naver user password: ")

    return user_id, user_pw


def get_search_query():
    """검색 질의어를 입력받아 돌려준다."""
    
    query = input("Enter your query: ")
    
    return query


def login(browser, user_id, user_pw):
    """네이버에 로그인한다."""

    browser.visit(LOGIN_URL)
    pause()

    id_elem = browser.find_by_css(ID_CSS)
    id_elem.fill(user_id)
    pause()

    pw_elem = browser.find_by_css(PW_CSS)
    pw_elem.fill(user_pw)
    pause()

    btn_elem = browser.find_by_css(LOGIN_BTN_CSS)
    btn_elem.click()
    pause()

    
def search(browser, query):
    """검색어를 입력하여 검색을 실행한다."""

    pause()
    search_elem = browser.find_by_css(QUERY_CSS)
    search_elem.fill(query)
    pause()

    btn_elem = browser.find_by_css(SEARCH_BTN_CSS)
    btn_elem.click()
    pause()

    
def print_html(browser):
    """현재 페이지의 HTML 텍스트를 화면에 표시한다."""

    print(browser.html)

    
def logout(browser):
    """네이버에서 로그아웃한다."""

    browser.visit(HOME_URL)
    pause()
    browser.visit(LOGOUT_URL)
    pause()

    
def pause():
    """정해진 만큼 쉰다."""

    time.sleep(SLEEP)

        
def main():
    """크롬 브라우저를 조종하여 네이버 검색 서비스를 이용한다."""

    user_id, user_pw = get_naver_user_id_pw()
    query = get_search_query()
    browser = splinter.Browser("chrome")

    login(browser, user_id, user_pw)
    search(browser, query)
    print_html(browser)
    logout(browser)

    browser.quit()
    
#
# 실행
#

main()

위의 소스에서 눈여겨 살펴볼 부분은 다음과 같다.

* `main()`
    - splinter 모듈의 Browse 클래스 객체를 이용할 때에 인자로 브라우저를 선택할 수 있다.
    - 현재 크롬과 파이어폭스, PhantomJS 드라이버가 지원된다.
    - `headless=True`를 지정하면서 크롬 드라이버를 사용하면 크롬이 헤드레스 모드로 실행된다.
* `longin()`
    - BeautifulSoup 객체의 `find()` 메소드에 대응하는 `find_by_css()` 메소드를 사용한다.
    - 사용자 행위를 흉내내기 위하여 `fill()` 메소드와 `click()` 메소드를 사용한다.

> 헤드레스 크롬이나 PhantomJS가 아닌 Firefox 등을 그래픽 환경이 없는 서버 환경에서 실행하려면 버츄얼 프레임버퍼(<https://en.wikipedia.org/wiki/Xvfb>)를 사용해야 한다.

# 트위터 API의 활용

140자 이내의 짧은 포스팅을 올릴 수 있는 마이크로 블로깅 서비스인 트위터는 오늘 대표적인 소셜 네트워크 서비스이다. 사용자들은 자신들의 다양한 일상사를 트위터를 통해 공유하므로 우리는 트윗의 분석을 통해 사람들의 선호, 경향, 유행 등을 분석할 수 있다.

생성되는 모든 실시간 트윗, 혹은 과거에 생성된 트윗을 분석하기 위해서는 소셜 데이터 공급 전문 회사인 지닙(GNIP, <https://gnip.com>)에서 제공하는 파이어호스 서비스를 이용해야 한다. 전수 데이터의 수집이 보장되지는 않지만 트위터에서 제공하는 API(https://dev.twitter.com) 통한 데이터 수집도 가능하다. 트위터에서는 여러 가지 형태의 데이터 수집을 지원하는 다수의 API를 제공하는데 우리는 검색 API와 스트리밍 API를 사용해 보자.

트위터 API는 HTTP 프로토콜에 기반한다. 그러므로 앞서 뉴스 기사의 수집에 사용한 Requests 모듈을 이용하여 트위터 API에 접근할 수 있다. 그러나 그보다는 사용하기에 편리하며 파이썬스러운(Pythonic) 라이브러리 모듈인 Twython(<https://github.com/ryanmcgrath/twython>)을 사용하는 것이 훨씬 좋은 방법이다. Twython은 아나콘다 파이썬에서 지원하지 않으므로 명령행에서 `pip`을 이용하여 설치해야 한다.

## 트위터 API 사용 권한 획득을 위한 인증
트위터 API는 무료로 사용할 수 있으나 사용량(access rate)에 제한이 있으며, 사용 권한을 획득해야만 사용할 수 있다. API 사용 권한은 다음과 같은 절차를 거쳐 얻는다.

### 트위터 앱 등록 사이트 접속

![트위터 앱 등록 사이트 접속](figs/twitter-apps-login.png)

먼저 트위터 앱 등록 사이트(<https://apps.twitter.com>)에 접속한다. 이미 트위터에 접속한 상태가 아니라면 트위터 서비스 접속을 하게 된다. 참고로 트위터 API를 사용하려면 사용자의 휴대 전화 번호가 프로필에 등록되어 있어야 한다.

### 트위터 앱 생성

![twitter-apps-create](figs/twitter-apps-create.png)

트위터 API를 사용하기 위한 권한은 API를 사용하는 애플리케이션별로 받아야 한다. 그러므로 최소한 하나의 애플리케이션을 생성해야 한다.

### 트위터 앱 기본 사항 입력

![트위터 앱 기본 사항 입력 상단](figs/twitter-apps-set-1.png)

![트위터 앱 기본 사항 입력 하단](figs/twitter-apps-set-2.png)

생성하는 애플리케이션의 기본 사항은 입력한다. 애플리케이션의 이름은 Twitter라는 단어는 포함할 수 없다. 애플리케이션 웹 사이트는 반드시 입력해야 하는데 특별히 입력할 사이트가 없다면 임시로 적절한 값을 입력하고 나중에 수정한다. 합의문에 동의하고 생성 단추를 누른다.

### Consumer Key와 Secret 확인

![Consumer Key와 Secret 확인](figs/twitter-apps-consumer.png)

이어서 표시되는 화면에서 `Keys and Access Tokens`를 누르면 애플리케이션의 고유 정보인 Consumer Key (API Key)와 Consumer Secret (API Secret)을 확인할 수 있다.

### Access Token과 Access Token Secret 생성

![Access Token과 Acess Token Secret 생성](figs/twitter-apps-access-create.png)

![Access Token과 Acess Token Secret 생성 결과](figs/twitter-apps-access-key.png)

같은 화면의 하단에서 Access Token을 생성하는 단추를 눌러 Access Token과 Access Token Secret을 발급받는다. 

## 검색 API를 이용한 트윗 수집
트위터 검색 API(<https://dev.twitter.com/rest/reference/get/search/tweets>)는 트위터 REST API의 일부로 과거 7일간 생성된 트윗의 일부를 대상으로 검색을 수행한다. 이 API는 사용자당 15분 동안 180 번의 이용이 가능한다. 읽기 권한만을 가지는 애플리케이션을 위한 권한을 사용하면 15분 동안 450 번의 이용이 가능하다.

Twython 모듈을 이용한 간단한 검색 API 사용례를 살펴보자.

In [ ]:
"""
트위터 검색 API(https://dev.twitter.com/rest/public/search)를 이용하여
트윗을 수집한다.
"""

import ujson
from twython import Twython


CONSUMER_KEY = "YOUR_CONSUMER_KEY"
CONSUMER_SECRET = "YOUR_CONSUMER_SECRET"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"


twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN,
                  ACCESS_TOKEN_SECRET)
result = twitter.search(q="문재인")

for status in result["statuses"]:
    tweet = ujson.dumps(status, ensure_ascii=False)
    tweet_cp949 = tweet.encode("cp949", errors="ignore")
    tweet = tweet_cp949.decode("cp949", errors="ignore")
    print(tweet)

위의 예제에서는 `search()` 메소드에 키워드 인자 `q`를 지정하여 텍스트 패턴 검색을 수행하였다. 트위터 API 문서에 설명된 대로 다양한 키워드 인자를 추가로 지정하여 복합적인 검색을 할 수도 있다. 검색 결과 데이터는 JSON 형식으로 얻어진다.

실행 시점에서 검색 결과가 얼마나 주어지느냐에 따라 스크립트의 실행이 금방 멈추기도 하고 조금 길게 진행되기도 한다. 생성된 결과는 주어진 검색 조건을 만족하는 트윗과 그에 대한 메타 정보가 붙은 JSON 라인들이다.

## 스트리밍 API를 이용한 트윗 수집
앞서 살펴본 검색 API는 최근 7일간 생성된 트윗의 일부를 대상으로 검색을 수행한다. 그런데 트위터 데이터의 유용성은 실시간 데이터에 접근할 때에 드러난다. 실시간으로 생성되는 트윗을 수집하려면 스트리밍 API(<https://dev.twitter.com/streaming/overview>)를 사용한다. 스트리밍 API는 트위터의 API 서버에 지속적인 접속(persistent connection)을 유지하면서 트윗이 생성될 때마다 지정한 필터링 조건에 맞는 트윗에 맞는 트윗을 골라준다. 다음은 스트리밍 API를 사용하는 예제 스크립트이다.

In [ ]:
"""
트위터 스트리밍 API(https://dev.twitter.com/streaming/overview)를 이용하여
트윗을 수집한다.
"""

import sys
import ujson
from twython import TwythonStreamer


CONSUMER_KEY = "YOUR_CONSUMER_KEY"
CONSUMER_SECRET = "YOUR_CONSUMER_SECRET"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"


class MyStreamer(TwythonStreamer):
    """트위터 스트리머 클래스"""

    def on_success(self, data):
        """스트리밍이 성공했을 때 수집한 데이터를 인쇄한다."""
        
        tweet = ujson.dumps(data, ensure_ascii=False)
        tweet_cp949 = tweet.encode("cp949", errors="ignore")
        tweet = tweet_cp949.decode("cp949")
        print(tweet)
        

    def on_error(self, status_code, data):
        """스트리밍 오류가 발생했을 때 상태 코드를 인쇄한다."""

        print(status_code)

        
def main():
    streamer = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, 
                          ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    streamer.statuses.filter(track="트와이스,에이핑크,마마무")
    
#
# 실행
#

main()

위의 예에서는 `filter()` 메소드의 키워드 인자 `track`에 쉼표로 구분된 복수 개의 키워드를 지정하여 트윗을 필터링한다. 키워드에 공백이 포함되면 두 키워드가 포함된 트윗을 필터링 한다. 즉, AND 조건으로 해석된다. 한 번에 400개의 키워드를 지정할 수 있으며 각 키워드는 60 바이트를 넘을 수 없다. 검색 API와 마찬가지로 필터링 조건을 지정하는 추가의 키워드 인자를 `filter()` 메소드에 지정할 수 있다.

스트리밍은 계속 대기하면서 트위터 API 서버로부터의 푸시를 기다리므로 특별한 오류가 발생하지 않는 한 계속 실행된다.
결과는 검색에 의한 수집의 경우와 같다.

## 참고
* 실용적인 대규모 웹 크롤링을 위해서는 수집 예절의 준수와 유효 시간 내 데이터 수집의 균형을 맞추는 데에 많은 고려를 해야 한다.
* 웹 크롤링에서는 예측하기 어려운 일이 많이 발생하므로 미리 대비책을 마련해 두어야 한다.
* 컨테이너, 가상 서버, 프록시 서버의 사용 등 다양한 기술적 인프라 지원이 필요하다.
* 대규모 크롤링을 위하여 scrapy(<http://scrapy.org>) 모듈의 사용을 고려하라.
* 자바스크립트 실행을 통한 HTML 렌더링을 위해 Splash(<https://github.com/scrapinghub/splash>), PhearJS(<http://phear.io>) 등의 사용을 고려하라.

# 참고 문헌
* 서진수 (2016) 『왕초보! 파이썬 배워 크롤러 DIY 하다!』. 더알음.
* Beltran, Aries (2013) _Getting Started with PhantomJS_. Packt Publishing.
* Collin, Mark (2015) _Mastering Selenium WebDriver_. Packt Publishing.
* Lawson, Richard (2015) _Web Scraping with Python_. Packt Publishing.
* Mitchell, Ryan (2015) _Web Scraping with Python_. O'Reilly Media.
* Russel, Matthew A. (2013) _Mining the Social Web_. 2nd ed. O'Reilly Media.

# 연습 문제
1. 위에서 보인 게시판 사이트 크롤링과 비슷한 방법으로 네이트 뉴스(<http://www.nate.com/?f=news>) 수집 코드를 작성하라. 사용자에게 수집 대상 날짜를 입력으로 받고 수집 대상 면종은 경제면으로 제한하자.
1. 웹진 사이트 <http://www.inven.co.kr/board/powerbbs.php?come_idx=2097&iskin=webzine>에서 댓글 수집을 위한 코드를 작성하라.